In [1]:
import numpy as np
import pandas as pd
import subprocess
import os
import re
import glob

In [2]:
def process_predictions(raw_pred, assay, outdir, filename, chrom_sizes='/project/deeprna/data/hg38/hg38.chrom.sizes'):
    out_bg = f'{outdir}/{filename}_{assay}.bedgraph'
    out_bw = f'{outdir}/{filename}_{assay}.bw'
    df = raw_pred[assay]
    df = df.to_dict()
    df_list = ['\t'.join(re.split(':|_', x)) + '\t' + str(y) for (x,y) in df.items()]
    with open(out_bg, 'w') as f:
        f.write('\n'.join(df_list))
    subprocess.run(["bedGraphToBigWig", out_bg, chrom_sizes, out_bw], check=True)
    return

In [5]:
assays = ['DNase', 'CTCF', 'H3K27ac', 'H3K4me3', 'H3K36me3', 'H3K27me3', 'H3K9me3', 'H3K4me1']
tissues = [124, 192, 213, 277, 323]
pred_dir = '/project/deeprna_data/epigept/corgi_benchmark/raw_predictions'
outdir = '/project/deeprna_data/epigept/corgi_benchmark/processed_predictions/'
for tissue in tissues:
    df_list = []
    for file in [pred_dir + f'/tissue_{tissue}_bed_{i}.csv' for i in range(4)]:
        df_list.append(pd.read_csv(file, index_col=[0]))
    if len(df_list) > 0:
        df = pd.concat(df_list)
        for assay in assays:
            process_predictions(df, assay, outdir, f'tissue_{tissue}')